<a href="https://colab.research.google.com/github/klydem11/AI_CW/blob/master/signature_recogontion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scipy==1.1.0

  Using cached https://files.pythonhosted.org/packages/a8/0b/f163da98d3a01b3e0ef1cab8dd2123c34aee2bafbb1c5bffa354cc8a1730/scipy-1.1.0-cp36-cp36m-manylinux1_x86_64.whl
ERROR: umap-learn 0.4.4 has requirement scipy>=1.3.1, but you'll have scipy 1.1.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement scipy==1.4.1; python_version >= "3", but you'll have scipy 1.1.0 which is incompatible.
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [2]:
import time
import requests
import zipfile
import io
import os
import glob
import pickle
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from scipy.misc import imread, imresize, imsave
from skimage.morphology import label
from skimage.measure import regionprops
from skimage.transform import resize
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import hog
import random
import natsort 
import warnings
warnings.filterwarnings('ignore')


In [3]:
r = requests.get('https://github.com/klydem11/AI_CW/raw/master/signatureRecognition.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [4]:
#paths for the training samples 
path_signature1 = './signatureRecognition/datasets/SVM/training/signature1/'
path_signature2 = './signatureRecognition/datasets/SVM/training/signature2/'
path_signature3 = './signatureRecognition/datasets/SVM/training/signature3/'

#get all the images in the above folders
#note that we are looking for specific extensions (jgg, bmp and png)
signature1_filenames = sorted([filename for filename in os.listdir(path_signature1) if (filename.endswith('.jpg') or filename.endswith('.png') or (filename.endswith('.bmp'))) ])
signature2_filenames = sorted([filename for filename in os.listdir(path_signature2) if (filename.endswith('.jpg') or filename.endswith('.png') or (filename.endswith('.bmp'))) ])
signature3_filenames = sorted([filename for filename in os.listdir(path_signature3) if (filename.endswith('.jpg') or filename.endswith('.png') or (filename.endswith('.bmp'))) ])

#add the full path to all the filenames
signature1_filenames = [path_signature1+filename for filename in signature1_filenames]
signature2_filenames = [path_signature2+filename for filename in signature2_filenames]
signature3_filenames = [path_signature3+filename for filename in signature3_filenames]

print('Number of training images -> signature1: ' + str(len(signature1_filenames)))
print('Number of training images -> signature2: ' + str(len(signature2_filenames)))
print('Number of training images -> signature3: ' + str(len(signature3_filenames)))

#create the list that will hold ALL the data and the labels
#the labels are needed for the classification task:
# 0 -> Bob
# 1 -> Susanne
# 2 -> Jeff   
data = []
labels = []

#fill the training dataset
# the flow is
# 1) load sample
# 2) resize it to (200,200) so that we have save size for all the images
# 3) get the HOG features of the resized image
# 4) save them in the data list that holds all the hog features
# 5) also save the label (target) of that sample in the labels list
for filename in signature1_filenames:
    #read the images
    image = imread(filename)
    #flatten it
    image = resize(image, (200,200))
    hog_features = hog(image, orientations=12, pixels_per_cell=(16, 16),
                cells_per_block=(1, 1))
    data.append(hog_features)
    labels.append(0)
print("Finished adding signature1s' samples to dataset")

for filename in signature2_filenames:
    image = imread(filename)
    image = resize(image, (200,200))
    hog_features = hog(image, orientations=12, pixels_per_cell=(16, 16),
                cells_per_block=(1, 1))
    data.append(hog_features)
    labels.append(1)
print("Finished adding signature2' samples to dataset")

for filename in signature3_filenames:
    image = imread(filename)
    image = resize(image, (200,200))
    hog_features = hog(image, orientations=12, pixels_per_cell=(16, 16),
                cells_per_block=(1, 1))
    data.append(hog_features)
    labels.append(2)
print("Finished adding signature3s' samples to dataset")

print(labels)

Number of training images -> signature1: 11
Number of training images -> signature2: 11
Number of training images -> signature3: 11
Finished adding signature1s' samples to dataset
Finished adding signature2' samples to dataset
Finished adding signature3s' samples to dataset
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]


In [5]:
print('Training the SVM')
r =  random.randint(1,30)
print(r)
#create the SVC
#clf = LinearSVC(dual=False,verbose=1)
#clf = svm.SVC(C=0.01, kernel='poly', degree=3, gamma=2)
clf = KNeighborsClassifier(n_neighbors=r)
#clf = DecisionTreeClassifier()
#train the svm
clf.fit(data, labels)

Training the SVM
2


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [6]:
#pickle it - save it to a file
pickle.dump( clf, open( "place.detector", "wb" ) )


In [7]:
def signature_detect(signature_image, random_number):
  #load the detector
  clf = pickle.load(open("place.detector","rb"))

  #now load a test image and get the hog features. 
  test_image = imread(signature_image)
  test_image = resize(test_image, (200,200))

  hog_features = hog(test_image, orientations=12, pixels_per_cell=(16, 16),
                cells_per_block=(1, 1))
  print("\nProcessing...")
  result_type = clf.predict(hog_features.reshape(1,-1))
  #print(result_type)
  name = ''
  if random_number == 1:
    name = 'Bob'
  elif random_number == 2:
    name = 'Susanne'
  elif random_number == 3:
    name = 'Jeff'
  
  print('\nSignature picked is signature ' + name)
  if result_type == 0:
    print("System: This is Bob's signature")
  elif result_type == 1:
    print("System: This is Susanne's signature")
  elif result_type == 2:
    print("System: This is Jeff's signature")
  else:
    print("I DON'T KNOW!")
  
  print("...done!!")
  return result_type



In [8]:
#getting genuine signature images at random to test (images here are not included in training)
positive = 0
answer_array = []
results_array = []
#testing genuine signature
for i in range(500):
  random_number1 = random.randint(1,3)
  random_number2 = random.randint(1,7)
  answer = random_number1
  genuine_signature_test = './signatureRecognition/datasets/SVM/testing/real_signature' + str(random_number1) +'/genuine-' + str(random_number2) + '.png'
  signature = signature_detect(genuine_signature_test, random_number1)
  result = signature + 1
  if answer == result:
    positive += 1
    answer_array.append(1)
    results_array.append(1)
  else:
    answer_array.append(0)
    results_array.append(0)


Processing...

Signature picked is signature Susanne
System: This is Susanne's signature
...done!!

Processing...

Signature picked is signature Susanne
System: This is Susanne's signature
...done!!

Processing...

Signature picked is signature Jeff
System: This is Jeff's signature
...done!!

Processing...

Signature picked is signature Susanne
System: This is Susanne's signature
...done!!

Processing...

Signature picked is signature Susanne
System: This is Susanne's signature
...done!!

Processing...

Signature picked is signature Bob
System: This is Bob's signature
...done!!

Processing...

Signature picked is signature Susanne
System: This is Susanne's signature
...done!!

Processing...

Signature picked is signature Bob
System: This is Bob's signature
...done!!

Processing...

Signature picked is signature Susanne
System: This is Susanne's signature
...done!!

Processing...

Signature picked is signature Jeff
System: This is Jeff's signature
...done!!

Processing...

Signature pi

In [10]:
print("Accuracy: " + str((positive/500)*100) + "%")
print("k= " + str(r))

Accuracy: 93.8%
k= 2
